In [22]:
import pandas as pd
import numpy as np
import json
import os
DATAFRAME_PATH = 'dataframes/playlists.h5'
def load_playlists(path):
    all_playlists = []
    for file in os.listdir("raw_data"):
        with open("raw_data/"+file) as f:
            js_slice = json.load(f)
            for playlist in js_slice["playlists"]:
                yield playlist
        print(f"Added playlists from {file}")
df = pd.DataFrame(load_playlists(DATAFRAME_PATH))
df.to_hdf(DATAFRAME_PATH, 'playlists')

Added playlists from mpd.slice.0-999.json
Added playlists from mpd.slice.1000-1999.json


D:\Anaconda3\lib\site-packages\pandas\core\generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['name', 'collaborative', 'tracks', 'description'], dtype='object')]

  pytables.to_hdf(


In [23]:
df = pd.read_hdf(DATAFRAME_PATH,'playlists')
df

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,woo,false,1995,1509062400,146,68,2,"[{'pos': 0, 'artist_name': 'blink-182', 'track...",13,29183178,24,NaN
1996,NEW YEARS,false,1996,1420070400,38,37,1,"[{'pos': 0, 'artist_name': 'Big Sean', 'track_...",3,9354755,34,NaN
1997,JESUS,false,1997,1508371200,40,28,2,"[{'pos': 0, 'artist_name': 'Elevation Worship'...",8,10940019,22,NaN
1998,yep,false,1998,1448236800,29,27,1,"[{'pos': 0, 'artist_name': 'Oasis', 'track_uri...",3,7260645,20,NaN
